#### Q.9
#####  On the MNIST dataset, train an SVM classifier. You'll need to use one-versus-the-rest to assign all 10 digits because SVM classifiers are binary classifiers. To accelerate up the process, you might want to tune the hyperparameters using small validation sets. What level of precision can you achieve?

In [3]:
import numpy as np
from sklearn import datasets
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler

In [4]:
#%% Get MNIST data and split in training and test sets

from sklearn.datasets import fetch_openml
mnist = fetch_openml('mnist_784', version=1, cache=True)

X = mnist["data"]
y = mnist["target"].astype(np.uint8)

X_train = X[:20000]
y_train = y[:20000]
X_test = X[20000:]
y_test = y[20000:]

C:\Users\pc\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\datasets\_openml.py:932: FutureWarning: The default value of `parser` will change from `'liac-arff'` to `'auto'` in 1.4. You can set `parser='auto'` to silence this warning. Therefore, an `ImportError` will be raised from 1.4 if the dataset is dense and pandas is not installed. Note that the pandas parser may return different data types. See the Notes Section in fetch_openml's API doc for details.
  warn(


In [5]:
X_train.shape

(20000, 784)

In [6]:
from sklearn.svm import LinearSVC
lin_clf=LinearSVC(random_state=42)
lin_clf.fit(X_train,y_train)



C:\Users\pc\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


LinearSVC(random_state=42)

In [7]:
#%% Train set accuracy score

from sklearn.metrics import accuracy_score

y_pred=lin_clf.predict(X_train)
accuracy_score(y_train,y_pred)



0.91375

In [8]:
#%% Train with scaled data

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train.astype(np.float32))
X_test_scaled = scaler.transform(X_test.astype(np.float32))

lin_clf = LinearSVC(random_state=42)
lin_clf.fit(X_train_scaled, y_train)

y_pred = lin_clf.predict(X_train_scaled)
accuracy_score(y_train, y_pred)

C:\Users\pc\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\svm\_base.py:1244: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


0.9444

In [9]:
#%% Train scaled data with SVC + RBF kernel

from sklearn.svm import SVC
svm_clf=SVC(gamma="scale")
svm_clf.fit(X_train_scaled[:10000],y_train[:10000])



SVC()

In [10]:
#%% Narrow down hyperparameters with randomized search + CV

from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import reciprocal, uniform

param_distributions = {'gamma': reciprocal(0.001, 0.1), 
                       'C': uniform(1, 10)}

In [11]:
rnd_search_cv = RandomizedSearchCV(svm_clf, param_distributions, n_iter=10, verbose=2, cv=3)
rnd_search_cv.fit(X_train_scaled[:1000], y_train[:1000])

rnd_search_cv.best_score_
rnd_search_cv.best_estimator_.fit(X_train_scaled, y_train)

y_pred = rnd_search_cv.best_estimator_.predict(X_train_scaled)
accuracy_score(y_train, y_pred)




Fitting 3 folds for each of 10 candidates, totalling 30 fits
[CV] END ...C=7.826878090832996, gamma=0.0074213356278525835; total time=   0.2s
[CV] END ...C=7.826878090832996, gamma=0.0074213356278525835; total time=   0.1s
[CV] END ...C=7.826878090832996, gamma=0.0074213356278525835; total time=   0.1s
[CV] END .....C=10.41986118984479, gamma=0.03276286993075358; total time=   0.2s
[CV] END .....C=10.41986118984479, gamma=0.03276286993075358; total time=   0.2s
[CV] END .....C=10.41986118984479, gamma=0.03276286993075358; total time=   0.2s
[CV] END .....C=4.794042957554583, gamma=0.06709370879304988; total time=   0.2s
[CV] END .....C=4.794042957554583, gamma=0.06709370879304988; total time=   0.1s
[CV] END .....C=4.794042957554583, gamma=0.06709370879304988; total time=   0.2s
[CV] END ...C=1.5704410096691759, gamma=0.038116822343643816; total time=   0.2s
[CV] END ...C=1.5704410096691759, gamma=0.038116822343643816; total time=   0.1s
[CV] END ...C=1.5704410096691759, gamma=0.038116

0.9953

In [12]:
#%% And now the test set

y_pred = rnd_search_cv.best_estimator_.predict(X_test_scaled)
accuracy_score(y_test, y_pred)

0.95788